In [1]:
#based https://github.com/jaara/ai_examples/blob/master/open_gym/MountainCar-v0.py
#Vlad Zhukov
import random, numpy, math
import gym
import tensorflow as tf
import numpy as np
import datetime
from time import gmtime, strftime

In [2]:
PROBLEM = 'MountainCar-v0'
env =  gym.make(PROBLEM)
high = env.observation_space.high
low = env.observation_space.low
print(low, high)
mean = (high + low) / 2
spread = abs(high - low) / 2
def normalize(s):
    return (s - mean) / spread

[2016-11-16 00:24:00,402] Making new env: MountainCar-v0


[-1.2  -0.07] [ 0.6   0.07]


In [ ]:
class My_model:
    def __init__(self, sess, name):
        self.sess = sess
        self.X = tf.placeholder(tf.float32, [None, 2], name=name + "X") 
        self.Y = tf.placeholder(tf.float32, [None, 2], name=name + "Y")
        self.w1 = tf.Variable(tf.truncated_normal([2, 256],stddev = 0.1), name=name + "w1")
#         with tf.Session() as sess:
#             tf.initialize_all_variables().run()
#             print (sess.run(self.w1).shape)
#             print (sess.run(self.X, feed_dict={self.X: [[1., 1.]]}).shape)
#             print (sess.run(tf.matmul(self.X, self.w1), feed_dict={self.X: [[1., 1.]]}).shape)
        self.b1 = tf.Variable(tf.zeros([256]), name=name + "b1")
        self.y1 = tf.nn.relu(tf.matmul(self.X, self.w1) + self.b1, name=name +"y1")
        
        self.w2 = tf.Variable(tf.truncated_normal([256, 2], stddev = 0.1), name=name+"w2")
        self.b2 = tf.Variable(tf.zeros([2]), name=name+"b2")
        self.y_model = tf.matmul(self.y1, self.w2) + self.b2#tf.nn.softmax(tf.matmul(self.y1, self.w2) + self.b2, name=name+"y_model")
        
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(self.y_model, self.Y), name=name + "cost")#tf.reduce_sum(tf.square(self.Y - self.y_model))
        
        self.train_op = tf.train.RMSPropOptimizer(0.1).minimize(self.cost, name=name + "train_op")
#         self.train_op = tf.train.GradientDescentOptimizer(0.1).minimize(self.cost, name=name + "train_op") 
    def predict(self, s):
#         print (self.sess.run(self.w1))
        return self.sess.run(self.y_model, feed_dict={self.X: s})
    def train(self, trX, trY):
    # you need to initialize variables (in this case just variable W)
#         print(self.sess.run(Y, feed_dict={X:trX, Y: trY}))
#         for i in range(100):
        for (x, y) in zip(trX, trY):
            self.sess.run(self.train_op, feed_dict={self.X: [x], self.Y: [y]})
#                 print(self.sess.run(self.cost, feed_dict={self.X:trX, self.Y:trY}))
#     def save(self):

class Brain:
    def __init__(self, stateCnt, actionCnt, sess):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.sess = sess
        self.model = My_model(sess, "model1")
        self.model_ = My_model(sess, "model2")
    def predict(self, s, target = False):
        if target:
#             return sess.run(model_.y_model, feed_dict={X: s})
            return self.model.predict(s)
        else:
#             return sess.run(model.y_model, feed_dict={X:s})
            return self.model_.predict(s)
    def print_w(self):
        print (self.sess.run([self.model.w, self.model_.w]))
    def train(self, trX, trY):
        self.model.train(trX, trY)
    def updateTargetModel(self):
        self.model_.w1.assign(self.model.w1)
        self.model_.w2.assign(self.model.w2)
        self.model_.b1.assign(self.model.b1)
        self.model_.b2.assign(self.model.b2)
    def save(self):
#         self.model.save()
#         self.model_.save()
        saver = tf.train.Saver()
        save_path = saver.save(self.sess, "saves/model1" + ".ckpt")
        print("Model saved in file: %s" % save_path)
    def load(self, sess, name):
        saver = tf.train.Saver()
        saver.restore(sess, name)

In [ ]:
# strftime("%Y-%m-%d %H:%M:%S", gmtime())

In [ ]:
with tf.Session() as sess:
#My_model(sess)
    model = Brain(2, 2, sess)
    init = tf.initialize_all_variables()
#     ckpt = tf.train.get_checkpoint_state("saves/model1.ckpt")
    ckpt = tf.train.get_checkpoint_state("saves/")
#     print (ckpt)
    model.load(sess, ckpt.model_checkpoint_path)
    sess.run(init)
#     print (model.print_w())
    
    print (model.predict([[1., 1.]], target = True))
#     model.predict([1, 1])
#     print (model.print_w())
#     model.train([[1, 1]],[[0.1, 0.5]])
#     model.save()
    print (model.predict([[1., 1.]], target = True))
#     print (model.print_w())

In [ ]:
class Memory:   # stored as ( s, a, r, s_ )
    samples = []

    def __init__(self, capacity):
        self.capacity = capacity

    def add(self, sample):
        self.samples.append(sample)        

        if len(self.samples) > self.capacity:
            self.samples.pop(0)

    def sample(self, n):
        n = min(n, len(self.samples))
        return random.sample(self.samples, n)
    
MEMORY_CAPACITY = 100000
BATCH_SIZE = 32
GAMMA = 0.99
MAX_EPSILON = 0.8
MIN_EPSILON = 0.1
LAMBDA = 0.0001      # speed of decay
UPDATE_TARGET_FREQUENCY = 10000

class Agent:
    def __init__(self, stateCnt, actionCnt, sess):
        self.stateCnt = stateCnt
        self.actionCnt = actionCnt
        self.brain = Brain(stateCnt, actionCnt, sess)
        self.memory = Memory(MEMORY_CAPACITY)
        
    def observe(self, sample):  # in (s, a, r, s_) format
        self.memory.add(sample)        

    def act(self, s):
        if random.random() < self.epsilon:
            return random.randint(0, self.actionCnt-1)
        else:
            return numpy.argmax(self.brain.predict(s))
        
    steps = 0
    epsilon = MAX_EPSILON
    def replay(self):
        if self.steps % UPDATE_TARGET_FREQUENCY == 0:
            self.brain.updateTargetModel()

        # slowly decrease Epsilon based on our eperience
        self.steps += 1
        self.epsilon = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * math.exp(-LAMBDA * self.steps)
        print(self.epsilon)
        batch = self.memory.sample(BATCH_SIZE)
        batchLen = len(batch)

        states = numpy.array([ o[0] for o in batch ])
        states_ = numpy.array([ ([0,0] if o[3] is None else o[3]) for o in batch ])

        p = agent.brain.predict(states)
        p_ = agent.brain.predict(states_, target=True)

        x = numpy.zeros((batchLen, self.stateCnt))
        y = numpy.zeros((batchLen, self.actionCnt))
        
        for i in range(batchLen):
            o = batch[i]
            s = o[0]; a = o[1]; r = o[2]; s_ = o[3]
            
            t = p[i]
            if s_ is None:
                t[a] = r
            else:
                # t[a] = r + GAMMA * numpy.amax(p_[i])          # single DQN
                t[a] = r + GAMMA * p_[i][ numpy.argmax(p[i]) ]  # double DQN

            x[i] = s
            y[i] = t            

        self.brain.train(x, y)


In [ ]:
def run(agent):
    s = normalize(env.reset())
#     print(s)
    R = 0
    step = 0
    memory = list()
    tf.initialize_all_variables().run()
    while True:
#         env.render()
#         print(s)
        a =agent.act([s]) 
        print (a)
    #     print (a)
    #     print (sess.run(w))
    #     print(a)
        a_ = 0
        if a == 1: 
            a_ = 2
        s_, r, done, info = env.step(a_)
        s_ = normalize(s_)
        if done: # terminal state
            s_ = None
        agent.observe((s, a, r, s_))
        s = s_
        R += r
        agent.replay()       
        step += 1
        if done:
            break

In [ ]:
stateCnt  = env.observation_space.shape[0]
actionCnt = 2
with tf.Session() as sess:
    env.monitor.start('MounrainCarRecords', force=True)
    agent = Agent(stateCnt, actionCnt, sess)
    while True:
        run(agent)
env.monitor.close()

In [ ]:
env.monitor.close()
env.close()


In [4]:
gym.upload('/Users/vlad/Documents/my_git_hub/DeepHack/RL/NN_agent/MounrainCarRecords', api_key='sk_4uOH7kK2R1SHIQlkRZotRw')

[2016-11-16 00:26:45,516] [MountainCar-v0] Uploading 3186 episodes of training data
[2016-11-16 00:27:01,505] [MountainCar-v0] Uploading videos of 13 training episodes (509084 bytes)
[2016-11-16 00:28:01,878] [MountainCar-v0] Creating evaluation object from /Users/vlad/Documents/my_git_hub/DeepHack/RL/NN_agent/MounrainCarRecords with learning curve and training video
[2016-11-16 00:28:03,037] 
****************************************************
You successfully uploaded your evaluation on MountainCar-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_tXdiYpNlQBaMhSqxKb4g

****************************************************
